# Projeto 1 - Ciência dos Dados

Nome: Enzo Qental

Nome: João Pedro Artese de Natal

Nome: João Pedro Ramos Fiorin

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\jprfi\OneDrive\Área de Trabalho\CDADOS\Projeto 1\CDADOS-P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Iphone1250.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,"Relevância (0-->Muito Irrelevante, 1-->Irrelevante, 2-->Relevante, 3--> Muito Relevante)"
0,"eu achava iphone caro,até pagar o preço de nun...",1
1,"novamente sem carregador, essa vida usuária de...",2
2,se fôssemos trocar de celular vocês ficariam c...,2
3,"@zaynikr mas gente, ela tem uns 3 telefones co...",2
4,esse face id do iphone que se eu tiver com a b...,3


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,"Relevância (0-->Muito Irrelevante, 1-->Irrelevante, 2-->Relevante, 3--> Muito Relevante)"
0,"voltei c iphone, que sdd que eu estava meu deus",1
1,@user928373891 sofreu por ser usuário de iphone,0
2,"so pq tem um iphone, é triste viu kkkkkjjjjj h...",0
3,vendo carregador de iphone (novinho e preço ne...,0
4,ela vive dando reply nos story usando iphone k...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Celular da Apple. Consideramos como uma postagem relevante quando o conteúdo se trata de alguma crítica ou análise sobre o produto, caso o contrário, consideramos como não relevante.

___
### Montando as funções de limpeza dos twits.

Criando as funções que irão limpar os nossos dados das duas planilhas(Treinamento e Teste).

In [22]:
train['Treinamento'] = train['Treinamento'].str.replace('[!-.:?\n*();/]', '')
train

,Treinamento,"Relevância (0-->Muito Irrelevante, 1-->Irrelevante, 2-->Relevante, 3--> Muito Relevante)"
0,eu achava iphone caroaté pagar o preço de nunc...,1
1,novamente sem carregador essa vida usuária de ...,2
2,se fôssemos trocar de celular vocês ficariam c...,2
3,@zaynikr mas gente ela tem uns 3 telefones com...,2
4,esse face id do iphone que se eu tiver com a b...,3
...,...,...
995,iphone xr 89 de bateria amigo está vendendo ma...,1
996,meu namorado me deu um iphone com a parte de t...,1
997,a saudade que to do meu iphone é surreal 😫,1
998,queria criar meu vsco logo mas iphone é tenso ...,1


In [23]:
test['Teste'] = test['Teste'].str.replace('[!-.:?\n*();/]', '')
test

,Teste,"Relevância (0-->Muito Irrelevante, 1-->Irrelevante, 2-->Relevante, 3--> Muito Relevante)"
0,voltei c iphone que sdd que eu estava meu deus,1
1,@user928373891 sofreu por ser usuário de iphone,0
2,so pq tem um iphone é triste viu kkkkkjjjjj ht...,0
3,vendo carregador de iphone novinho e preço neg...,0
4,ela vive dando reply nos story usando iphone k...,0
...,...,...
245,como instalar o whatsapp gb watusi no seu iph...,0
246,minha vida mudou depois que o vinicius desativ...,0
247,a bateria do meu celular tá parecendo de iphon...,2
248,@narcisa narcisa me dá um iphone 🥺,0


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**